# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 64 new papers today
          22 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/21 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2303.00008


extracting tarball to tmp_2303.00008...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.00009


extracting tarball to tmp_2303.00009...

...

extracting tarball to tmp_2303.00626...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.00687


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.00626/1Paper.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:312: LatexWarning: Latex injecting: 'eriappend' from 'tmp_2303.00626/eriappend.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:312: LatexWarning: Latex injecting: 'ericontent' from 'tmp_2303.00626/ericontent.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2303.00687...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.00012-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.00012) | **Planetary Population Synthesis and the Emergence of Four Classes of  Planetary System Architectures**  |
|| Alexandre Emsenhuber, Christoph Mordasini, <mark>Remo Burn</mark> |
|*Appeared on*| *2023-03-02*|
|*Comments*| *Invited review accepted for publication in EPJ+, Focus Point on Environmental and Multiplicity Effects on Planet Formation by guest editors G. Lodato and C.F. Manara*|
|**Abstract**| Planetary population synthesis is a tool to understand the physics of planetary system formation. It builds on a model that includes a multitude of physical processes. The outcome can be statistically compared with exoplanet observations. Here, we review the population synthesis method and then use one population to explore how different planetary system architectures emerge and which conditions lead to their formation. The systems can be classified into four main architectures: Class I of near-in situ compositionally ordered terrestrial and ice planets, Class II of migrated sub-Neptunes, Class III of mixed low-mass and giant planets, broadly similar to the Solar System, and Class IV of dynamically active giants without inner low-mass planets. These four classes exhibit distinct typical formation pathways and are characterised by certain mass scales. Class I systems form from the local accretion of planetesimals followed by a giant impact phase, and the final planet masses correspond to the `Goldreich mass'. Class II systems form when planets reach the `equality mass' (equal accretion and migration timescales) before the dispersal of the gas disc, but not large enough to allow for rapid gas accretion. Giant planets form when the `equality mass' allows for rapid gas accretion while the planet are migrating, i.e. when the critical core mass is reached. The main discriminant of the four classes is the initial mass of solids in the disc, with contributions from the lifetime and mass of the gas disc. The breakdown into classes allows to better understand which physical processes are dominant. Comparison with observations reveals certain differences to the actual population, pointing at limitation of theoretical understanding. For example, the overrepresentation of synthetic super Earths and sub-Neptunes in Class I causes these planets to be found at lower metallicities than in observations. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.00044-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.00044) | **The Initial Mass Function and Other Stellar Properties Across the Core  of the Hydra I Cluster**  |
|| Ilaria Lonoce, Wendy Freedman, <mark>Anja Feldmeier-Krause</mark> |
|*Appeared on*| *2023-03-02*|
|*Comments*| *32 pages, 18 figures. Accepted for publication in ApJ*|
|**Abstract**| The Hydra I cluster offers an excellent opportunity to study and compare the relic old stellar populations in the core of its two brightest galaxies. In addition, the differing kinematics of the two galaxies allows a test of the local validity of general scaling relations. In this work we present a direct comparison employing full spectral fitting of new high-quality long-slit optical and NIR spectroscopic data. We retrieve age, metallicity and 19 elemental abundances out to about 12 kpc within each galaxy, as well as the IMF in their central regions. Our results suggest that the inner 5 kpc region of both galaxies, despite their different masses, formed at the same time and evolved with a similar star formation time-scale and chemical enrichment, confirming their early formation in the cluster build up. Only the overall metallicity and IMF radial profiles show differences connected with their different velocity dispersion profiles. The radial trend of the IMF positively correlates with both [Z/H] and velocity dispersion. While the trends of the IMF with metallicity agree with a global trend for both galaxies, the trends with the velocity dispersion exhibit differences. The outer regions show signs of mixed stellar populations with large differences in chemical content compared to the centers, but with similar old ages. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.00489-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.00489) | **The miniJPAS survey quasar selection II: Machine learning classification  with photometric measurements and uncertainties**  |
|| Natália V.N. Rodrigues, et al. -- incl., <mark>Héctor Vázquez Ramió</mark> |
|*Appeared on*| *2023-03-02*|
|*Comments*| *16 pages, 15 figures, published by MNRAS*|
|**Abstract**| Astrophysical surveys rely heavily on the classification of sources as stars, galaxies or quasars from multi-band photometry. Surveys in narrow-band filters allow for greater discriminatory power, but the variety of different types and redshifts of the objects present a challenge to standard template-based methods. In this work, which is part of larger effort that aims at building a catalogue of quasars from the miniJPAS survey, we present a Machine Learning-based method that employs Convolutional Neural Networks (CNNs) to classify point-like sources including the information in the measurement errors. We validate our methods using data from the miniJPAS survey, a proof-of-concept project of the J-PAS collaboration covering $\sim$ 1 deg$^2$ of the northern sky using the 56 narrow-band filters of the J-PAS survey. Due to the scarcity of real data, we trained our algorithms using mocks that were purpose-built to reproduce the distributions of different types of objects that we expect to find in the miniJPAS survey, as well as the properties of the real observations in terms of signal and noise. We compare the performance of the CNNs with other well-established Machine Learning classification methods based on decision trees, finding that the CNNs improve the classification when the measurement errors are provided as inputs. The predicted distribution of objects in miniJPAS is consistent with the putative luminosity functions of stars, quasars and unresolved galaxies. Our results are a proof-of-concept for the idea that the J-PAS survey will be able to detect unprecedented numbers of quasars with high confidence. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.00481-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.00481) | **A detailed view of low-frequency quasi-periodic oscillation in the  broadband 0.2-200 keV with Insight-HXMT and NICER**  |
|| X. Ma, et al. -- incl., <mark>L. Zhang</mark>, <mark>S.N. Zhang</mark>, <mark>S. Zhang</mark> |
|*Appeared on*| *2023-03-02*|
|*Comments*| **|
|**Abstract**| We report the X-ray timing results of the black hole candidate MAXI J1820+070 during its 2018 outburst using the Hard X-ray Modulation Telescope (Insight-HXMT) and Neutron Star Interior Composition Explorer Mission (NICER) observations. Low frequency quasi-periodic oscillations (LFQPOs) are detected in the low/hard state and the hard intermediate state, which lasted for about 90 days. Thanks to the large effective area of Insight-HXMT at high energies and NICER at low energies, we are able to present the energy dependence of the LFQPO characteristics and phase lags from 0.2 keV to 200 keV, which has never been explored by previous missions. We find that the centroid frequency of the LFQPOs do not change significantly with energy, while the full width at half maximum (FWHM) and fractional rms show a complex evolution with energy. The LFQPO phase lags at high energies and low energies show consistent energy-dependence relations taking the ~2 keV as reference. Our results suggest that the LFQPOs from high energy come from the LT precession of the relativistic jet, while the low-energy radiation is mainly from the perpendicular innermost regions of the accretion disk. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.00518-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.00518) | **Detector bandwidth and polarisation switching rates: spectrophotometric  observations of the Sun by the Birmingham Solar Oscillations Network (BiSON)**  |
|| S. J. Hale, et al. -- incl., <mark>G. R. Davies</mark> |
|*Appeared on*| *2023-03-02*|
|*Comments*| *6 pages, 9 figures, 1 table. Accepted by RAS Techniques and Instruments: 2023 March 1*|
|**Abstract**| The Birmingham Solar Oscillations Network (BiSON) observes acoustic oscillations of the Sun. The dominant noise source is caused by fluctuations of Earth's atmosphere, and BiSON seeks to mitigate this effect by combining multiple rapid observations in alternating polarisation states. Current instrumentation uses bespoke Pockels-effect cells to select the polarisation state. Here, we investigate an alternative off-the-shelf solution, a liquid crystal retarder, and discuss the potential impact of differences in performance. We show through electrical simulation of the photodiode-based detectors, and assessment of both types of polarisation device, that although the switching rate is slower the off-the-shelf LCD retarder is a viable replacement for a bespoke Pockels-effect cell. The simplifications arising from the use of off-the-shelf components allows easier and quicker instrumentation deployment. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.00626-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.00626) | **WALLABY Pre-Pilot Survey: Radio Continuum Properties of the Eridanus  Supergroup**  |
|| J. A. Grundy, et al. -- incl., <mark>K. Lee-Waddell</mark> |
|*Appeared on*| *2023-03-02*|
|*Comments*| *26 pages, 14 figures, Accepted for publication at PASA 23/2/2023, Full catalogues and underlying data available at: this https URL*|
|**Abstract**| We present the highest resolution and sensitivity $\sim1.4\,$GHz continuum observations of the Eridanus supergroup obtained as a part of the Widefield ASKAP L-band Legacy All-sky Blind surveY (WALLABY) pre-pilot observations using the Australian Square Kilometer Array Pathfinder (ASKAP). We detect 9461 sources at 1.37 GHz down to a flux density limit of $\sim0.1$ mJy at $6.1''\times 7.9''$ resolution with a mean root-mean-square (RMS) of 0.05 mJy/beam. We find that the flux scale is accurate to within 5% (compared to NVSS at 1.4 GHz). We then determine the global properties of eight Eridanus supergroup members, which are detected in both radio continuum and neutral hydrogen (HI) emission, and find that the radio-derived star formation rates (SFRs) agree well with previous literature. Using our global and resolved radio continuum properties of the nearby Eridanus galaxies, we measure and extend the infrared-radio correlation (IRRC) to lower stellar masses and inferred star formation rates than before. We find the resolved IRRC to be useful for: 1) discriminating between AGN and star-forming galaxies (SFGs); 2) identifying background radio sources; and 3) tracing the effects of group environment pre-processing in NGC 1385. We find evidence for tidal interactions and ram-pressure stripping in the HI, resolved spectral index and IRRC morphologies of NGC 1385. There appears to be a spatial coincidence (in projection) of double-lobed radio jets with the central HI hole of NGC 1367. The destruction of polycyclic aromatic hydrocarbons (PAHs) by merger-induced shocks may be driving the observed WISE W3 deficit observed in NGC 1359. Our results suggest that resolved radio continuum and IRRC studies are excellent tracers of the physical processes that drive galaxy evolution and will be possible on larger sample of sources with upcoming ASKAP radio continuum surveys. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.00687-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.00687) | **Ground calibration of Gamma-Ray Detectors of GECAM-C**  |
|| Chao Zheng, et al. -- incl., <mark>Da-Li Zhang</mark>, <mark>Yan-Qiu Zhang</mark>, <mark>Dong-Ya Guo</mark>, <mark>Gang Li</mark>, <mark>Xin-Qiao Li</mark>, <mark>Yan-Guo Li</mark>, <mark>Mao-Shun Li</mark>, <mark>Li-Ming Song</mark>, <mark>Fan Zhang</mark>, <mark>Peng Zhang</mark>, <mark>Zhen Zhang</mark>, <mark>Yi Zhao</mark> |
|*Appeared on*| *2023-03-02*|
|*Comments*| *preliminary version, will be updated soon*|
|**Abstract**| As a new member of GECAM mission, GECAM-C (also named High Energy Burst Searcher, HEBS) was launched onboard the SATech-01 satellite on July 27th, 2022, which is capable to monitor gamma-ray transients from $\sim$ 6 keV to 6 MeV. As the main detector, there are 12 gamma-ray detectors (GRDs) equipped for GECAM-C. In order to verify the GECAM-C GRD detector performance and to validate the Monte Carlo simulations of detector response, comprehensive on-ground calibration experiments have been performed using X-ray beam and radioactive sources, including Energy-Channel relation, energy resolution, detection efficiency, SiPM voltage-gain relation and the non-uniformity of positional response. In this paper, the detailed calibration campaigns and data analysis results for GECAM-C GRDs are presented, demonstrating the excellent performance of GECAM-C GRD detectors. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.00537-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.00537) | **The performance of SiPM-based gamma-ray detector (GRD) of GECAM-C**  |
|| <mark>Dali Zhang</mark>, et al. -- incl., <mark>Xinqiao Li</mark>, <mark>Dongya Guo</mark>, <mark>Yanqiu Zhang</mark>, <mark>Xiaoyun Zhao</mark>, <mark>Xiaobo Li</mark>, <mark>Peng Zhang</mark>, <mark>Zhen Zhang</mark>, <mark>Yanguo Li</mark>, <mark>Bing Li</mark>, <mark>Fan Zhang</mark> |
|*Appeared on*| *2023-03-02*|
|*Comments*| *18 pages, 16 figures*|
|**Abstract**| As a new member of GECAM mission, the GECAM-C (also called High Energy Burst Searcher, HEBS) is a gamma-ray all-sky monitor onboard SATech-01 satellite, which was launched on July 27th, 2022 to detect gamma-ray transients from 6 keV to 6 MeV, such as Gamma-Ray Bursts (GRBs), high energy counterpart of Gravitational Waves (GWs) and Fast Radio Bursts (FRBs), and Soft Gamma-ray Repeaters (SGRs). Together with GECAM-A and GECAM-B launched in December 2020, GECAM-C will greatly improve the monitoring coverage, localization, as well as temporal and spectral measurements of gamma-ray transients. GECAM-C employs 12 SiPM-based Gamma-Ray Detectors (GRDs) to detect gamma-ray transients . In this paper, we firstly give a brief description of the design of GECAM-C GRDs, and then focus on the on-ground tests and in-flight performance of GRDs. We also did the comparison study of the SiPM in-flight performance between GECAM-C and GECAM-B. The results show GECAM-C GRD works as expected and is ready to make scientific observations. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2303.00012.md
    + _build/html/tmp_2303.00012/./sys_dsm_ptm.png
    + _build/html/tmp_2303.00012/./sys_dsm.png
    + _build/html/tmp_2303.00012/./classI.png
    + _build/html/tmp_2303.00012/./am_timescales_sim0074.png
exported in  _build/html/2303.00044.md
    + _build/html/tmp_2303.00044/./figures/ALF_ageZHimfALPHA_Hydra_FINAL.png
    + _build/html/tmp_2303.00044/./figures/Literature_comparison_KIN_PAPER.png
    + _build/html/tmp_2303.00044/./figures/ALF_ALLelements_Hydra_moreMask_FINALproc.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>



<div id="title">

# Planetary Population Synthesis and the Emergence of Four Classes of Planetary System Architectures

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.00012-b31b1b.svg)](https://arxiv.org/abs/2303.00012)<mark>Appeared on: 2023-03-02</mark> - _Invited review accepted for publication in EPJ+, Focus Point on Environmental and Multiplicity Effects on Planet Formation by guest editors G. Lodato and C.F. Manara_

</div>
<div id="authors">

Alexandre Emsenhuber, Christoph Mordasini, <mark>Remo Burn</mark>

</div>
<div id="abstract">

**Abstract:** Planetary population synthesis is a helpful tool to understand the physics of planetary system formation. It builds on a global model, meaning that the model has to include a multitude of physical processes. The outcome can be statistically compared with exoplanet observations. Here, we review the population synthesis method and then use one population computed using the Generation III Bern model to explore how different planetary system architectures emerge and which conditions lead to their formation. The emerging systems can be classified into four main architectures: Class I of near-in situ compositionally ordered terrestrial and ice planets, Class II of migrated sub-Neptunes, Class III of mixed low-mass and giant planets, broadly similar to the Solar System, and Class IV of dynamically active giants without inner low-mass planets. These four classes exhibit distinct typical formation pathways and are characterised by certain mass scales. We find that Class I forms from the local accretion of planetesimals followed by a giant impact phase, and the final planet masses correspond to what is expected from such a scenario, the `Goldreich mass'. Class II, the migrated sub-Neptune systems form when planets reach the `equality mass' where accretion and migration timescales are comparable before the dispersal of the gas disc, but not large enough to allow for rapid gas accretion. Giant planets form when the `equality mass' allows for gas accretion to proceed while the planet are migrating, i.e., when the critical core mass is reached. The main discriminant of the four classes is the initial mass of solids in the disc, with contributions from the lifetime and mass of the gas disc. The distinction between mixed Class III systems and Class IV dynamically-active giants is in part due to the stochastic nature of dynamical interactions, such as scatterings between giant planets, rather than the initial conditions only. The breakdown of system into classes allows to better interpret the outcome of a complex model and understand which physical processes are dominant. Comparison with observations reveals differences to the actual population, pointing at limitation of theoretical understanding. For example, the overrepresentation of synthetic super Earths and sub-Neptunes in Class I systems causes these planets to be found at lower metallicities than in observations.

</div>

<div id="div_fig1">

<img src="tmp_2303.00012/./sys_dsm_ptm.png" alt="Fig14.1" width="50%"/><img src="tmp_2303.00012/./sys_dsm.png" alt="Fig14.2" width="50%"/>

**Figure 14. -** System architecture as function of the initial solids mass content in the disc $M_{\rm p,ini}$ and the total mass (accounting for solids and gas) in the final planets per system $M_{\rm sys}$(left) and a histogram of initial solids mass only (right). In the left panel, the three dashed diagonal lines again denote the conversion efficiency from disc solids to planets, with the value given next to them. In the right panel, the small horizontal lines show the extent of each bin. (*fig:sys_dsm_ptm*)

</div>
<div id="div_fig2">

<img src="tmp_2303.00012/./classI.png" alt="Fig4" width="100%"/>

**Figure 4. -** Final architecture of four example systems in Class I. Coloured dots show the position in the mass-semimajor axis plane of different planet composition types as indicated in the legend. The three numbers in the top left give the initial conditions [M/H], initial gas disc mass [\si{\msun}], and initial mass of planetesimals [\si{\mearth}]. The systems are ordered according to the latter. Horizontal black bars indicate the periastron to apastron distance, reflecting  orbital eccentricity. Grey full circles show the last position of protoplanets that were lost. The ones lost by accretion onto the star are put on the left y-axis of each panel and those ejected on the right one. The remaining grey points correspond to protoplanets that were accreted by another more massive protoplanet. The point then shows the mass and semi-major axis just before this accretion event. (*fig:class1examples*)

</div>
<div id="div_fig3">

<img src="tmp_2303.00012/./am_timescales_sim0074.png" alt="Fig5" width="100%"/>

**Figure 5. -** Example of formation tracks and final architecture of a system that belong to Class II. In this class, the governing mass scale in the inner system is linked to orbital migration and is thus given by the saturation mass $M_{\rm sat}$ where planets are released from migration traps (green lines, evaluated at $t=0$ and $t=\SI{0.1}{\mega\year}$) and/or the equality mass $M_{\rm{eq}}$ where the timescales of oligarchic growth and type I orbital migration  are equal (cyan line). (*fig:class2*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand$
$\newcommand$
$\newcommand{\afk}[1]{\textcolor{purple}{\uline{#1}}}$
$\newcommand{\afks}[1]{\textcolor{purple}{\sout{#1}}}$
$\newcommand{\il}[1]{\textcolor{cyan}{\uline{#1}}}$
$\newcommand{\ils}[1]{\textcolor{cyan}{\sout{#1}}}$
$\newcommand{\wf}[1]{\textcolor{blue}{\uline{#1}}}$
$\newcommand{\wfs}[1]{\textcolor{blue}{\sout{#1}}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand$
$\newcommand$
$\newcommand{\afk}[1]{\textcolor{purple}{\uline{#1}}}$
$\newcommand{\afks}[1]{\textcolor{purple}{\sout{#1}}}$
$\newcommand{\il}[1]{\textcolor{cyan}{\uline{#1}}}$
$\newcommand{\ils}[1]{\textcolor{cyan}{\sout{#1}}}$
$\newcommand{\wf}[1]{\textcolor{blue}{\uline{#1}}}$
$\newcommand{\wfs}[1]{\textcolor{blue}{\sout{#1}}}$</div>



<div id="title">

# The initial mass function and other stellar properties across the core of the Hydra I cluster\footnote{This paper includes data gathered with the 6.5 meterMagellan Telescopes located at Las Campanas Observatory, Chile.}

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.00044-b31b1b.svg)](https://arxiv.org/abs/2303.00044)<mark>Appeared on: 2023-03-02</mark> - _32 pages, 18 figures. Accepted for publication in ApJ_

</div>
<div id="authors">

Ilaria Lonoce, W. L. Freedman, <mark>A. Feldmeier-Krause</mark>

</div>
<div id="abstract">

**Abstract:** The Hydra I cluster offers an excellent opportunity to study and compare the relic old stellar populations in the core of its two brightest galaxies. In addition, the differing kinematics of the two galaxies allows a test of the local validity of general scaling relations. In this work we present a direct comparison employing full spectral fitting of new high-quality long-slit optical and NIR spectroscopic  data. We retrieve age, metallicity and 19 elemental abundances out to $\sim12$ kpc within each galaxy, as well as the IMF in their central regions.Our results suggest that the inner $\sim5$ kpc region of both galaxies, despite their different masses, formed at the same time and evolved with a similar star formation time-scale and chemical enrichment, confirming their early formation in the cluster build up. Only the overall metallicity and IMF radial profiles show differences connected with their different velocity dispersion profiles. The radial trend of the IMF positively correlates with both [ Z/H ] and $\sigma$ . While the trends of the IMF with metallicity agree with a global trend for both galaxies, the trends with the velocity dispersion exhibit differences. The outer regions show signs of mixed stellar populations with large differences in chemical content compared to the centers, but with similar old ages.

</div>

<div id="div_fig1">

<img src="tmp_2303.00044/./figures/ALF_ageZHimfALPHA_Hydra_FINAL.png" alt="Fig8" width="100%"/>

**Figure 8. -** \small{Age, metallicity and IMF slope trends across the Hydra I cluster center as retrieved with ALF. West is on the left, East is on the right. The bottom panel
shows the derived mismatch parameter $\alpha_r$.
Dash-dotted horizontal tan lines in the IMF and $\alpha_r$ panels outline the corresponding values for a Salpeter IMF, dashed those for a Kroupa IMF. Open diamonds refer to regions where the results are less robust due to lower S/N, velocity dispersion broadening and/or lack of important features. } (*fig:ageZimf*)

</div>
<div id="div_fig2">

<img src="tmp_2303.00044/./figures/Literature_comparison_KIN_PAPER.png" alt="Fig11" width="100%"/>

**Figure 11. -** \small{Velocity dispersion (upper panel) and radial velocity (lower panel) profiles as retrieved with ALF (orange).
Green triangles and red circles show an indication of the trends retrieved by the works of  ([Richtler, et. al (2011)](https://ui.adsabs.harvard.edu/abs/2011A&A...531A.119R))  and  ([Hilker, et. al (2018)](https://ui.adsabs.harvard.edu/abs/2018A&A...619A..70H)) , respectively, as they appear in figure $6$ of  ([Hilker, et. al (2018)](https://ui.adsabs.harvard.edu/abs/2018A&A...619A..70H))  for the case of position angle $108$\degr.  Typical error bars are shown on the right of both panels.
} (*fig:kinematics*)

</div>
<div id="div_fig3">

<img src="tmp_2303.00044/./figures/ALF_ALLelements_Hydra_moreMask_FINALproc.png" alt="Fig9" width="100%"/>

**Figure 9. -** \small{Similar to Figure \ref{fig:ageZimf} but for all the retrieved elemental abundances.} (*fig:elements*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

217  publications in the last 7 days.
	 _build/html/2303.00044.md
	 _build/html/2303.00012.md
	 _build/html/2302.14137.md
	 _build/html/2302.12805.md
	 _build/html/2302.10943.md
	 _build/html/2302.10528.md
	 _build/html/2302.10008.md
	 _build/html/2302.08962.md
	 _build/html/2302.08628.md
	 _build/html/2302.07916.md
	 _build/html/2302.07880.md
	 _build/html/2302.07497.md
	 _build/html/2302.07277.md
	 _build/html/2302.07256.md
	 _build/html/2302.07234.md
	 _build/html/2302.07057.md
	 _build/html/2302.05694.md
	 _build/html/2302.05465.md
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.03039.md
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> daily </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.
